<a href="https://colab.research.google.com/github/Catofood/PixelMind/blob/Linear-and-NN-structure/Linear_layer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
import numpy as np
import math

# ---------- Функции активации ----------
def ReLU(x):
    return np.maximum(0, x)

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def softmax(x: np.array):
    # Вычитаем максимум
    # (дополнительно можно ограничить значения)
    x_max = np.max(x, axis=1, keepdims=True)
    x_stable = x - x_max
    # Можно добавить огран. например: x_stable = np.clip(x_stable, -15, 15)

    exps = np.exp(x_stable)
    sums = np.sum(exps, axis=1, keepdims=True)

    # sums может быть inf
    # Чтобы избежать деления на 0:
    sums = np.clip(sums, 1e-9, None)
    return exps / sums


# ---------- Функции потерь ----------
def categorical_cross_entropy_loss(y_pred: np.array, y_true: np.array):
    """
    y_pred: вероятности (после softmax), shape=(batch_size, n_classes)
    y_true: one-hot метки, shape=(batch_size, n_classes)

    Возвращает (loss_value, grad),
      где grad = dL/dy_pred той же формы, что y_pred.
    """
    eps = 1e-7
    # 1) Считаем кросс-энтропию
    N = y_pred.shape[0]
    loss_value = - np.mean(np.sum(y_true * np.log(y_pred + eps), axis=1))

    # 2) Градиент по y_pred
    #    dL/dy_pred = -1/N * (y_true / y_pred)
    grad = - (y_true / (y_pred + eps)) / N
    return loss_value, grad


def mse_loss(y_pred, y_true):
    N = y_pred.shape[0]
    loss_value = np.mean((y_pred - y_true)**2)
    grad = 2.0 * (y_pred - y_true) / N  # dL/dy_pred
    return loss_value, grad

In [19]:
# -- Производные активаций --
def d_relu(z):
    return (z > 0).astype(float)

def d_sigmoid(z):
    sig = 1 / (1 + np.exp(-z))
    return sig * (1 - sig)

In [24]:
class Linear:
    def __init__(self, input_size, output_size, activation_func=None, activation_deriv=None):
        self.n_in = input_size
        self.n_out = output_size

        # Инициализация весов
        self.weights = np.random.randn(self.n_in, self.n_out) * 0.01
        self.bias = np.zeros((1, self.n_out))  # иногда лучше инициализировать нулями

        self.activation_func = activation_func if activation_func else lambda x: x
        self.activation_deriv = activation_deriv

        # Для backward сохраняем промежуточное
        self.input = None
        self.z = None
        self.output = None

    def forward(self, x: np.array):
        """
        x: (batch_size, n_in)
        """
        self.input = x
        # Линейная часть
        self.z = np.dot(x, self.weights) + self.bias
        # Активация
        self.output = self.activation_func(self.z)
        return self.output

    def backward(self, d_out: np.array, learning_rate: float):
        """
        d_out = dL/d(output этого слоя).

        Возвращаем dL/d(input этого слоя).
        """
        # dZ = d_out * activation'(z) (покомпонентное)
        if self.activation_deriv is not None:
            dZ = d_out * self.activation_deriv(self.z)
        else:
            dZ = d_out

        # dW = X^T * dZ
        dW = np.dot(self.input.T, dZ)
        # db = сумма dZ по batch
        db = np.sum(dZ, axis=0, keepdims=True)
        # dX = dZ * W^T
        dX = np.dot(dZ, self.weights.T)

        # Обновляем параметры
        self.weights -= learning_rate * dW
        self.bias -= learning_rate * db

        return dX

In [26]:
class NeuralNetwork:
    def __init__(self, layers=None, loss_func=None, lr=0.01):
        self.layers = layers if layers else []
        self.loss_func = loss_func  # должна возвращать (loss_value, dL/dy_pred)
        self.lr = lr

    def add_layer(self, layer: Linear):
        self.layers.append(layer)

    def forward(self, x: np.array):
        for layer in self.layers:
            x = layer.forward(x)
        return x

    def backward(self, x: np.array, y_true: np.array):
        """
        1) Forward
        2) Считаем loss и dOut
        3) backward (reverse layers)
        """
        # forward
        y_pred = self.forward(x)
        # loss + dL/dy_pred
        loss_value, d_out = self.loss_func(y_pred, y_true)

        # идём с конца
        for layer in reversed(self.layers):
            d_out = layer.backward(d_out, self.lr)

        return loss_value

In [31]:
# ======= Пример: бинарная классификация (2 класса) =======
if __name__ == "__main__":
    np.random.seed(42)

    N = 200
    d_in = 2
    X = np.random.randn(N, d_in)

    # Придумаем закономерность: класс 1, если x0 + x1 > 0, иначе класс 0
    y_int = np.where(X[:,0] + X[:,1] > 0, 1, 0)

    y_true = np.zeros((N, 2))
    y_true[np.arange(N), y_int] = 1

    # 1) Первый слой (2->8), ReLU
    layer1 = Linear(2, 8, activation_func=ReLU, activation_deriv=d_relu)
    # 2) Второй слой (8->2), softmax
    #    Удобнее softmax сделать отдельным слоем, но сделаем "в лоб":
    #    linear forward, а activation_func=softmax
    #    (Производную softmax напрямую мы не пишем —
    #     используем формулу dL/dy_pred=-1/N*y_true/y_pred и предполагаем y_pred=softmax)
    layer2 = Linear(8, 2, activation_func=softmax, activation_deriv=None)

    # Создаём модель
    model = NeuralNetwork(
        layers=[layer1, layer2],
        loss_func=categorical_cross_entropy_loss,
        lr=0.01  # шаг побольше, чтобы обучение шло заметнее
    )

    # Обучаем несколько эпох
    n_epochs = 100
    for epoch in range(n_epochs):
        loss_val = model.backward(X, y_true)
        if epoch % 10 == 0:
            print(f"Epoch {epoch}, loss = {loss_val:.4f}")

    # Проверим результат (предсказанные вероятности)
    preds = model.forward(X)  # shape=(N, 2)
    # Преобразуем к меткам класса: argmax по оси 1
    pred_labels = np.argmax(preds, axis=1)
    true_labels = np.argmax(y_true, axis=1)

    accuracy = np.mean(pred_labels == true_labels)
    print(f"\nFinal loss: {loss_val:.4f}, accuracy: {accuracy*100:.2f}%")

    # Посмотрим на первые 5 примеров
    print("Some predictions:")
    for i in range(5):
        print(f"X[i]={X[i]},  pred={preds[i]},  pred_class={pred_labels[i]},  true_class={true_labels[i]}")

Epoch 0, loss = 0.6931
Epoch 10, loss = 0.6930
Epoch 20, loss = 0.6930
Epoch 30, loss = 0.6930
Epoch 40, loss = 0.6929
Epoch 50, loss = 0.6929
Epoch 60, loss = 0.6928
Epoch 70, loss = 0.6927
Epoch 80, loss = 0.6926
Epoch 90, loss = 0.6925

Final loss: 0.6923, accuracy: 90.50%
Some predictions:
X[i]=[ 0.49671415 -0.1382643 ],  pred=[0.49983303 0.50016697],  pred_class=1,  true_class=1
X[i]=[0.64768854 1.52302986],  pred=[0.49937062 0.50062938],  pred_class=1,  true_class=1
X[i]=[-0.23415337 -0.23413696],  pred=[0.50032982 0.49967018],  pred_class=0,  true_class=0
X[i]=[1.57921282 0.76743473],  pred=[0.49937905 0.50062095],  pred_class=1,  true_class=1
X[i]=[-0.46947439  0.54256004],  pred=[0.50020806 0.49979194],  pred_class=0,  true_class=1


In [32]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

def one_hot_encode(y, num_classes):
    """
    Превращает массив целых меток, например [0,1,2,1,0],
    в one-hot, например [[1,0,0],[0,1,0],[0,0,1],...].
    """
    y_encoded = np.zeros((y.shape[0], num_classes))
    y_encoded[np.arange(y.shape[0]), y] = 1
    return y_encoded